# TODO Compared with from scratch standard training and attacks 

In [17]:
%load_ext autoreload
%autoreload 2


PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#TransferLearning of ATTACK --> 
# VS From Scratch
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from finetune_framework import *

import warnings
warnings.filterwarnings('ignore')

In [19]:
model_name = "resnet"
num_classes = 10
feature_extract = False

_, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
train_dataset, val_dataset, train_loader, val_loader = CIFAR10(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"

Files already downloaded and verified
Files already downloaded and verified


##### Prevent overfiting  load from pt file

In [20]:
model_load = torch.load("./cifar_resnet_scratch.pt")
model_scratch = torch.load("./cifar_resnet_scratch.pt")


In [21]:
model_load

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
params_to_update = model_load.parameters()

feature_extract = False

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_load.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_scratch_adv_pgd_10.pth'
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_scratch_PGD_10")
pgd_attack = generate_adv(model_load, "pgd")

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [23]:
train(model_load, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 10, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/10]	
Epoch [1/10]	 Training Loss: 2.0562123133398384	 lr: 0.001
Epoch [1/10]	 Validation Loss: 17.87442981140523	 lr: 0.001
Epoch [1/10]	
Epoch [2/10]	 Training Loss: 1.7340256417803752	 lr: 0.001
Epoch [2/10]	 Validation Loss: 13.489908761615995	 lr: 0.001
Epoch [2/10]	
Epoch [3/10]	 Training Loss: 1.6726093176380752	 lr: 0.001
Epoch [3/10]	 Validation Loss: 15.941841717007794	 lr: 0.001
Epoch [3/10]	
Epoch [4/10]	 Training Loss: 1.6331837945582006	 lr: 0.001
Epoch [4/10]	 Validation Loss: 15.829559579680238	 lr: 0.001
Epoch [4/10]	
Epoch [5/10]	 Training Loss: 1.6057883257146381	 lr: 0.001
Epoch [5/10]	 Validation Loss: 14.965834895266763	 lr: 0.001
Epoch [5/10]	
Epoch [6/10]	 Training Loss: 1.5791728139838295	 lr: 0.001
Epoch [6/10]	 Validation Loss: 14.379839571216438	 lr: 0.001
Epoch [6/10]	
Epoch [7/10]	 Training Loss: 1.5702302800419996	 lr: 0.001
Epoch [7/10]	 Validation Loss: 17.246113656442375	 lr: 0.001
Epoch [7/10]	
Epoch [8/10]	 Training Loss: 1.5551093408213856	 

In [24]:
print("the clean accuracy is ")
test(val_loader, model_load)

the clean accuracy is 
Accuracy of the network on the 10000 test images: 32 %


In [25]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, model_load, "cuda")

PGD
[Model loaded]
Acc: 64.36 %
adv model


In [26]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, model_load, "cuda")

fgsm
[Model loaded]
Acc: 64.35 %
adv model


In [27]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, model_load, "cuda")

nifgsm
[Model loaded]
Acc: 64.19 %
adv model


In [28]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, model_load, "cuda")

vmifgsm
[Model loaded]
Acc: 63.91 %
adv model
